In [1]:
import re
from functools import partial
from multiprocessing import Pool, cpu_count

import nltk
import pandas as pd
import statsmodels.api as sm
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm

import logging

import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Functions

In [2]:
logging.basicConfig(
    filename="tfidf.log",
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

def multiprocess_tasks(func, tasks, data, njobs=cpu_count()):
    wrapper = partial(func, df=data)
    res = []
    with Pool(processes=njobs) as p:
        n = len(tasks)
        with tqdm(total=n) as pbar:
            for i, v in enumerate(p.imap_unordered(wrapper, tasks)):
                res.append(v)
                pbar.update()
    return res


def r_squared_adj(data, group, dummy=True, var_name="palt"):
    y = data[var_name]
    if dummy:
        x = data[group]
    else:
        x = pd.get_dummies(data[group], drop_first=True)
    x = sm.add_constant(x)
    mod = sm.OLS(y, x).fit()
    return mod.rsquared_adj.round(2)


def match_token(text, token):
    tokens = tokenize_only(text)
    if token in tokens:
        return 1
    else:
        return 0


def token_power(token, df, desc_column="award_description"):
    df[token] = df[desc_column].apply(match_token, token=token)
    r2 = r_squared_adj(df, token)
    return {token: r2}


def tokenize_and_stem(text):
    stemmer = SnowballStemmer("english")
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [
        word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)
    ]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search("[a-zA-Z]", token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [
        word.lower()
        for sent in nltk.sent_tokenize(text)
        for word in nltk.word_tokenize(sent)
    ]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search("[a-zA-Z]", token):
            filtered_tokens.append(token)
    return filtered_tokens

## Process award description of IRS contracts 

In [3]:
import glob
import os

# read in all other contract
dfs = []
for fp in glob.glob('data/treasury/*.zip'):
    if '2019' in fp or '2020' in fp:
        df = pd.read_csv(fp, low_memory=False, dtype={'modification_number':str, 'parent_award_agency_id':str, 'awarding_sub_agency_code':str, 'funding_sub_agency_code':str})
        df['fy'] = os.path.basename(fp)[2:6]
        dfs.append(df)
dat = pd.concat(dfs)

# IRS contracts only
dat = dat.loc[(dat.modification_number=='0')&((dat.parent_award_agency_id=='2050')|(dat.awarding_sub_agency_code=='2050')|(dat.funding_sub_agency_code=='2050'))]

dat['action_date'] = pd.to_datetime(dat['action_date'])
dat['solicitation_date'] = pd.to_datetime(dat['solicitation_date'])
dat['palt'] = (dat['action_date'] - dat['solicitation_date']).dt.days
dat = dat.loc[dat.palt.notnull()].reset_index(drop=True)

# # select 95%tile for outlier cutoff
palt = dat.loc[dat.palt<322].reset_index(drop=True)
print(dat.shape)

(924, 284)


## Process award description of Splunk contracts 

In [4]:
# read in splunk
splunk = pd.read_csv('data/USA3_Splunk_slim.csv')
print(splunk.shape)

max_df=0.8
max_features=2000
min_df=0.04
use_idf=True
tokenizer=tokenize_and_stem
ngram_range=(1, 3)
random_state=1111

domain_stopwords = ["OT", "CL", "CT", "option", "closeout", "contractor", "contract", "fund", "funding", "funds", "exercise", "service", "services", "cyrbyme", "support", "igf"]

en_stopwords = nltk.corpus.stopwords.words("english")
stopwords = en_stopwords + [i.lower() for i in domain_stopwords]
stopwords = tokenize_and_stem(" ".join(stopwords))

logging.info(
    f"max_df={max_df}, max_features={max_features}, min_df={min_df}, use_idf={use_idf}, ngram_range={ngram_range}, domain_stopwords={domain_stopwords}"
)

# define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(
    max_df=max_df,
    max_features=max_features,
    min_df=min_df,
    stop_words=stopwords,
    use_idf=use_idf,
    tokenizer=tokenizer,
    ngram_range=ngram_range,
)

tfidf_matrix = tfidf_vectorizer.fit_transform(splunk["award_description"].tolist())

terms = tfidf_vectorizer.get_feature_names()

logging.info(f"{len(terms)} features extracted: {terms}")

splunk_terms = tfidf_matrix.toarray()

print(terms)

(171, 135)
['annual', 'brand', 'brand name', 'enterpris', 'enterpris licens', 'hardwar', 'licens', 'mainten', 'name', 'order', 'profession', 'purchas', 'purpos', 'renew', 'requir', 'secur', 'server', 'softwar', 'softwar licens', 'splunk enterpris', 'splunk enterpris licens', 'splunk licens', 'splunk profession', 'splunk softwar', 'splunk softwar licens', 'standard', 'term']


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exerci'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## Measure and rank order similarity between all IRS contracts and Splunk contracts

In [5]:
# splunk center
np.random.seed(random_state)
km = KMeans(n_clusters=1)
km.fit(tfidf_matrix)

splunk_center = km.cluster_centers_

# project all contract award_description onto tf-idf feature space
dat_vec = tfidf_vectorizer.transform(dat["award_description"].tolist()).toarray()
dat[terms] = pd.DataFrame(dat_vec)

# calculate cosine similarity to the splunk center
cs = cosine_similarity(splunk_center, dat_vec)
dat["Cosine similarity"] = cs.squeeze()
dat = dat.sort_values(by="Cosine similarity", ascending=False)

dat[["award_description", "Cosine similarity"]].head()

,award_description,Cosine similarity
788,PROSCI ENTERPRISE CHANGE MANAGEMENT SITE LICEN...,0.593653
605,BLACKBERRY ENTERPRISE MOBILITY LICENSE PROGRAM,0.593653
669,VERITAS INFOSCALE SOFTWARE SUITE LICENSES AND ...,0.543943
411,IRS ORACLE ENTERPRISE LICENSE AGREEMENT (ELA) ...,0.543002
410,ENTERPRISE PHYSICAL ACCESS CONTROL SYSTEM (EPA...,0.541499


In [6]:
# output
dat[["award_id_piid","award_description"] + terms + ["Cosine similarity"]].to_csv('output/splunk_similarity_tfidf.csv', index=False)